# Chords

The idea of this is to: 
- A) scrape popular song chords, 
- B) mathematically infer their actual musical mode, 
- C) create database of modes, and 
- D) Analyze

Final products could be:

 a chord-writer, 

 an addition to the spotify playlist algos (Better feature that they dont have?)
 
 etc...

## Step 1 - Scraping Chordify

Chordify gives chords in a very easy grid format if you do the following:
1. Go to the song of your choice (https://chordify.net/chords/ed-sheeran-songs/shape-of-you-chords)
2. Select "Print" OR Go to "Overview" and Scrape

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [134]:
urlz = ['https://chordify.net/chords/ed-sheeran-songs/shape-of-you-chords', 
'https://chordify.net/chords/the-fray-songs/how-to-save-a-life-2-chords', 
'https://chordify.net/chords/the-weeknd-songs/blinding-lights-2-chords?version=youtube:fHI8X4OXluQ']

In [135]:
# Put it all together
driver = webdriver.Chrome()

for u in urlz:
    driver.get(u)
    button_xpath = '/html/body/div[4]/main/div[3]/div/div[2]/div/div[4]/div/div/div/button[2]'
    driver.find_element(by=By.XPATH, value=button_xpath).click()
    elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, button_xpath)))
    src = driver.page_source
    parser = BeautifulSoup(src, "lxml")
    table = parser.find("div", attrs = {"id":"chords"})
    row_names = table.findAll('span') #
    table2 = parser.find("div", attrs = {"class":"w688stb"})
    key = table2.findAll('span')
    chordz1 = str(row_names)
    chordz1 = chordz1.replace('<span class=', '')
    chordz1 = chordz1.replace('"chord-label icon-rest"></span>', '')
    chordz1 = chordz1.replace('"bass-label"></span>', '')
    chordz1 = chordz1.replace('></span>', '')
    chordz1 = chordz1.replace('"chord-label label-', '')
    chordz1 = chordz1.replace('"', '')
    chordz1 = chordz1.replace(' ', '')
    chordz2 = chordz1.split(',')
    better_list = []
    for x in chordz2:
        if x != '':
            better_list.append(x)
    bl_df = pd.DataFrame(better_list)
    bl_df = bl_df.rename(columns = {0 : 'chords'})
    song_chords = bl_df.groupby('chords')['chords'].count()
    key = str(key)
    key2 = key.split(',')
    key3 = pd.DataFrame(key2)
    key4 = key3.loc[key3[0].str.contains('span class="label-')]
    key4 = key4.replace('"', '')
    key4 = key4.replace('"></span>', '')
    key5 = key4.iloc[[2]]
    final_key = key5[0].astype(str).str[20:]
    final_key = final_key.str.replace('"></span>', '')
    song_chords = song_chords.append(final_key)
    title = str(u)[28:].replace('/', '_')
    title = title + '.xlsx'
    song_chords.to_excel('songs/' + title)

## To Do Next:

Add ability to search chordify.net for songs (auto) to build a url list to connect the billboard project with this one